In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### LLM에 도구 바인딩 (Binding Tools)

LLM 모델이 도구(tool) 를 호출할 수 있으려면 chat 요청을 할 때 모델에 도구 스키마(tool schema) 를 전달해야 합니다.

In [3]:
from bs4 import BeautifulSoup
from langchain.agents import tool
import requests
import re

In [4]:
@tool
def get_word_length(word: str) -> int:          # 단어의 길이를 리턴하는 도구
    """Returns the length of a word."""

    return len(word)

In [5]:
@tool
def add_function(a: float, b: float) -> float:  # 두 수의 합을 리턴하는 도구
    """Adds two numbers together."""
    
    return a + b

In [6]:
@tool
def naver_news_crawl(news_url: str) -> str:                 # 네이버 뉴스기사를 크롤링하여 본문 내용을 리턴하는 도구
    """Crawls a 네이버(naver.com) news article and returns the body content."""

    response = requests.get(news_url)                       # HTTP GET 요청

    if response.status_code == 200:                         # 응답 200. 성공
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find('h2', id='title_area').get_text()
        content = soup.find('div', id='contents').get_text()    
        
        cleaned_title = re.sub(r'\n{2, }', '\n', title)
        cleaned_content = re.sub(r'\n{2, }', '\n', content)
    else:
        print(f'HTTP 요청 실패. {response.status_code}')
    
    return f'{cleaned_title}\n{cleaned_content}'

In [7]:
tools = [get_word_length, add_function, naver_news_crawl]  # 여러 개의 도구를 한꺼번에 바인딩 위해서 리스트에 넣기

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

llm_with_tools = llm.bind_tools(tools)                      # 도구 바인딩(llm_with_tools는 도구를 가지고 있다.)

In [9]:
answer = llm_with_tools.invoke('랭체인 이라는 단어의 글자 길이는?')     # chain 실행의 결과는 어떤 도구를 사용해야 하는지와 도구에 전달 되는 인자와 같은 판단이다.

In [10]:
answer

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_s6UNcIGpeksfmoZksz2Jj0xR', 'function': {'arguments': '{"word":"랭체인"}', 'name': 'get_word_length'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 117, 'total_tokens': 136, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f0676530-ffa0-4ad3-8a04-7415495c3b3b-0', tool_calls=[{'name': 'get_word_length', 'args': {'word': '랭체인'}, 'id': 'call_s6UNcIGpeksfmoZksz2Jj0xR', 'type': 'tool_call'}], usage_metadata={'input_tokens': 117, 'output_tokens': 19, 'total_tokens': 136, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoni

In [11]:
answer.content

''

In [12]:
answer.additional_kwargs

{'tool_calls': [{'id': 'call_s6UNcIGpeksfmoZksz2Jj0xR',
   'function': {'arguments': '{"word":"랭체인"}', 'name': 'get_word_length'},
   'type': 'function'}],
 'refusal': None}

In [13]:
answer = llm_with_tools.invoke('랭체인 이라는 단어의 글자 길이는?').tool_calls      # chain 실행의 결과는 어떤 도구를 사용해야 하는지와 도구에 전달 되는 인자와 같은 판단이다.

In [14]:
answer 

[{'name': 'get_word_length',
  'args': {'word': '랭체인'},
  'id': 'call_DAWKSH8kUFCfXt4UDg5HXNqp',
  'type': 'tool_call'}]

In [15]:
print(answer[0]['name'])    # 실행했을 때 사용할 도구의 이름
print(answer[0]['args']  )  # 실행했을 때 도구에 전달되는 인자

get_word_length
{'word': '랭체인'}


In [16]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# llm_with_tools으로 도구를 실행해서 나온 결과(실행할 때 사용할 도구의 이름, 도구에 전달되는 인자)를 도구 목록이 정의 된 JsonOutputToolsParser(tools=tools)에 연결
# 도구 바인딩 + 도구 파서
chain = llm_with_tools | JsonOutputToolsParser(tools=tools)     

tool_call_result = chain.invoke('랭체인 이라는 단어의 글자 길이는?')

In [17]:
print(tool_call_result)                 # chain 실행의 결과는 어떤 도구를 사용해야 하는지와 도구에 전달 되는 인자와 같은 판단이다. 
print('==' * 50)

single_result = tool_call_result[0]     # 첫 번째 도구 호출 결과
print(single_result['type'])            # 도구 이름
print(single_result['args'])            # 도구 인자

[{'args': {'word': '랭체인'}, 'type': 'get_word_length'}]
get_word_length
{'word': '랭체인'}


In [19]:
print(tool_call_result[0]['type'])      # 도구 이름 (chain을 호출 했을 때)
print(tools[0].name )                   # 도구 이름 (tools 의 도구 이름)

get_word_length
get_word_length


In [20]:
def execute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수

    :param tool_call_results: 도구 호출 결과 리스트
    :param tools: 사용 가능한 도구 리스트
    """

    for tool_call_result in tool_call_results:      # chain을 호출 했을 때의 결과를 가지고 있는 tool_call_results(도구 호출 결과 리스트)를 반복해서 
                                                    # 사용할 도구의 이름과 도구에 전달되는 인자를 추출한다. 
        
        tool_name = tool_call_result['type']        # 도구의 이름 
        tool_args = tool_call_result['args']        # 도구에 전달되는 인자

        print(f'tool_name : {tool_name}')
        print(f'tool_args : {tool_args}')
        
        matching_tool = None                        

        for tool in tools:                              # tools 리스트에 정의 된 도구 이름과 일치하는 첫 번째 도구를 찾습니다.
            if tool.name == tool_name:
                matching_tool = tool
                break
        
        if matching_tool:                               # 일치하는 도구를 찾은 경우
            result = matching_tool.invoke(tool_args)    # 해당 도구를 실행. 여기서 직접 호출해서 얻은 결과가 최종 실행결과.

            print(f'[실행도구] {tool_name} [Argument] {tool_args}')     # 실행결과 출력
            print(f'[실행결과] {result}')
        else:                                           # 일치하는 도구를 찾지 못한 경우
            print(f'경고: {tool_name}에 해당하는 도구를 찾을 수 없습니다.')

In [21]:
execute_tool_calls(tool_call_result)

tool_name : get_word_length
tool_args : {'word': '랭체인'}
[실행도구] get_word_length [Argument] {'word': '랭체인'}
[실행결과] 3
